<a href="https://colab.research.google.com/github/1804054Miraz/My_Thesis_Work/blob/main/Training_Testing_Validation_of_Mean_of_Gamma_Beta_and_KatzFD_with_full_channels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
%%capture
import numpy as np
import os
import pandas as pd
import statistics as st
import scipy.stats as scst
from scipy.stats import skew
from scipy.stats import kurtosis
from scipy.stats import entropy
from scipy.stats import normaltest
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import time
from sklearn.ensemble import BaggingClassifier

In [109]:
path = '/content/drive/MyDrive/Dataset/Features/merged_beta_gamma_a_c.csv'
merged_beta_gamma_a_c = pd.read_csv(path)
# merged_beta_gamma_a_c=merged_beta_gamma_a_c.drop(columns=['X_PS','X_PS_2','Y_PS','Y_PS_2','nd_PS', 'nd_PS_2'], axis=False)
merged_beta_gamma_a_c

,FP1_PS,FP2_PS,F7_PS,F8_PS,AF1_PS,AF2_PS,FZ_PS,F4_PS,F3_PS,FC6_PS,...,PO8_PS_2,FCZ_PS_2,POZ_PS_2,OZ_PS_2,P2_PS_2,P1_PS_2,CPZ_PS_2,nd_PS_2,Y_PS_2,label
0,49.125112,75.401561,55.085147,236.773934,24.574587,34.914910,9.395342,15.433209,21.208462,9.854879,...,31.407003,0.639929,3.175360,7.257440,1.171091,1.727645,0.303052,5.626485,6.090333,addicted
1,31.662543,140.818732,107.251892,299.075377,22.347470,53.286704,9.112230,29.795814,17.161432,20.283656,...,31.837468,0.481474,1.360249,3.002968,0.452312,0.873387,0.169722,4.693551,3.472028,addicted
2,33.981550,119.618734,69.705635,404.815675,24.381613,75.511201,9.212180,28.717643,24.864757,23.216713,...,39.507001,0.403271,1.391440,2.908869,0.790318,1.077771,0.227142,6.266331,3.612442,addicted
3,39.844284,101.587017,85.574482,491.630726,27.440661,55.378706,14.178997,42.917834,23.212199,26.417654,...,30.478540,0.450695,1.121842,2.162650,0.753562,0.999099,0.180620,4.211706,6.950914,addicted
4,51.900094,54.809999,113.999959,336.252881,29.900918,53.569130,11.944623,34.313907,17.342946,30.299428,...,55.854653,1.011451,3.663678,5.646425,1.918202,2.340282,0.437771,7.413797,7.107321,addicted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7617,56.398217,64.924927,102.732291,88.351802,48.641285,65.179016,36.246569,81.446588,56.383928,53.427282,...,5.024580,0.227139,3.721454,4.377724,1.713244,1.729774,0.453551,5.866098,3.061931,normal
7618,16.385750,19.103398,36.256479,29.794270,14.541851,18.748430,13.719778,28.332358,13.236784,30.672314,...,3.040292,0.398802,3.180998,2.880482,2.019003,1.853624,0.785453,2.408728,3.512361,normal
7619,53.679662,66.320784,87.491928,83.587297,46.040212,55.310880,36.804056,60.752408,44.290746,36.999455,...,2.070562,0.132006,1.298873,1.126842,0.855509,0.550932,0.247462,1.363607,3.663259,normal
7620,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,normal


In [110]:
merged_beta_gamma_a_c = merged_beta_gamma_a_c.drop(7620)
merged_beta_gamma_a_c = merged_beta_gamma_a_c.reset_index(drop=True)
merged_beta_gamma_a_c

,FP1_PS,FP2_PS,F7_PS,F8_PS,AF1_PS,AF2_PS,FZ_PS,F4_PS,F3_PS,FC6_PS,...,PO8_PS_2,FCZ_PS_2,POZ_PS_2,OZ_PS_2,P2_PS_2,P1_PS_2,CPZ_PS_2,nd_PS_2,Y_PS_2,label
0,49.125112,75.401561,55.085147,236.773934,24.574587,34.914910,9.395342,15.433209,21.208462,9.854879,...,31.407003,0.639929,3.175360,7.257440,1.171091,1.727645,0.303052,5.626485,6.090333,addicted
1,31.662543,140.818732,107.251892,299.075377,22.347470,53.286704,9.112230,29.795814,17.161432,20.283656,...,31.837468,0.481474,1.360249,3.002968,0.452312,0.873387,0.169722,4.693551,3.472028,addicted
2,33.981550,119.618734,69.705635,404.815675,24.381613,75.511201,9.212180,28.717643,24.864757,23.216713,...,39.507001,0.403271,1.391440,2.908869,0.790318,1.077771,0.227142,6.266331,3.612442,addicted
3,39.844284,101.587017,85.574482,491.630726,27.440661,55.378706,14.178997,42.917834,23.212199,26.417654,...,30.478540,0.450695,1.121842,2.162650,0.753562,0.999099,0.180620,4.211706,6.950914,addicted
4,51.900094,54.809999,113.999959,336.252881,29.900918,53.569130,11.944623,34.313907,17.342946,30.299428,...,55.854653,1.011451,3.663678,5.646425,1.918202,2.340282,0.437771,7.413797,7.107321,addicted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7616,28.486375,39.398962,72.987461,28.648872,26.222865,31.824731,32.457546,31.413086,28.013546,28.027574,...,2.408939,0.174153,0.867184,1.234990,0.535141,0.633024,0.261792,1.058754,6.116856,normal
7617,56.398217,64.924927,102.732291,88.351802,48.641285,65.179016,36.246569,81.446588,56.383928,53.427282,...,5.024580,0.227139,3.721454,4.377724,1.713244,1.729774,0.453551,5.866098,3.061931,normal
7618,16.385750,19.103398,36.256479,29.794270,14.541851,18.748430,13.719778,28.332358,13.236784,30.672314,...,3.040292,0.398802,3.180998,2.880482,2.019003,1.853624,0.785453,2.408728,3.512361,normal
7619,53.679662,66.320784,87.491928,83.587297,46.040212,55.310880,36.804056,60.752408,44.290746,36.999455,...,2.070562,0.132006,1.298873,1.126842,0.855509,0.550932,0.247462,1.363607,3.663259,normal


In [111]:
merged_beta_gamma_a_c = merged_beta_gamma_a_c.replace(0, merged_beta_gamma_a_c.mean(numeric_only=True))

In [112]:
merged_beta_gamma_a_c

,FP1_PS,FP2_PS,F7_PS,F8_PS,AF1_PS,AF2_PS,FZ_PS,F4_PS,F3_PS,FC6_PS,...,PO8_PS_2,FCZ_PS_2,POZ_PS_2,OZ_PS_2,P2_PS_2,P1_PS_2,CPZ_PS_2,nd_PS_2,Y_PS_2,label
0,49.125112,75.401561,55.085147,236.773934,24.574587,34.914910,9.395342,15.433209,21.208462,9.854879,...,31.407003,0.639929,3.175360,7.257440,1.171091,1.727645,0.303052,5.626485,6.090333,addicted
1,31.662543,140.818732,107.251892,299.075377,22.347470,53.286704,9.112230,29.795814,17.161432,20.283656,...,31.837468,0.481474,1.360249,3.002968,0.452312,0.873387,0.169722,4.693551,3.472028,addicted
2,33.981550,119.618734,69.705635,404.815675,24.381613,75.511201,9.212180,28.717643,24.864757,23.216713,...,39.507001,0.403271,1.391440,2.908869,0.790318,1.077771,0.227142,6.266331,3.612442,addicted
3,39.844284,101.587017,85.574482,491.630726,27.440661,55.378706,14.178997,42.917834,23.212199,26.417654,...,30.478540,0.450695,1.121842,2.162650,0.753562,0.999099,0.180620,4.211706,6.950914,addicted
4,51.900094,54.809999,113.999959,336.252881,29.900918,53.569130,11.944623,34.313907,17.342946,30.299428,...,55.854653,1.011451,3.663678,5.646425,1.918202,2.340282,0.437771,7.413797,7.107321,addicted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7616,28.486375,39.398962,72.987461,28.648872,26.222865,31.824731,32.457546,31.413086,28.013546,28.027574,...,2.408939,0.174153,0.867184,1.234990,0.535141,0.633024,0.261792,1.058754,6.116856,normal
7617,56.398217,64.924927,102.732291,88.351802,48.641285,65.179016,36.246569,81.446588,56.383928,53.427282,...,5.024580,0.227139,3.721454,4.377724,1.713244,1.729774,0.453551,5.866098,3.061931,normal
7618,16.385750,19.103398,36.256479,29.794270,14.541851,18.748430,13.719778,28.332358,13.236784,30.672314,...,3.040292,0.398802,3.180998,2.880482,2.019003,1.853624,0.785453,2.408728,3.512361,normal
7619,53.679662,66.320784,87.491928,83.587297,46.040212,55.310880,36.804056,60.752408,44.290746,36.999455,...,2.070562,0.132006,1.298873,1.126842,0.855509,0.550932,0.247462,1.363607,3.663259,normal


In [113]:
# Assuming 'new_dataframe' is your DataFrame
X = merged_beta_gamma_a_c.iloc[:, :-1]
y = merged_beta_gamma_a_c.iloc[:, -1]

# Encode the target variable if it's categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Splitting the data into training and test sets (80% training, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [115]:

# Creating SVM classifier pipeline
svm_classifier = make_pipeline(StandardScaler(), SVC())

# Training SVM classifier on the training set
svm_classifier.fit(X_train, y_train)

# Making predictions on the test set using SVM
svm_predictions_test = svm_classifier.predict(X_test)

# Evaluating SVM model on the test set
svm_accuracy_test = accuracy_score(y_test, svm_predictions_test)
svm_precision_test = precision_score(y_test, svm_predictions_test)
svm_recall_test = recall_score(y_test, svm_predictions_test)
svm_f1_test = f1_score(y_test, svm_predictions_test)

# Performing K-fold cross-validation on the training set for SVM
svm_predictions_cv = cross_val_predict(svm_classifier, X_train, y_train, cv=KFold(n_splits=10, shuffle=True, random_state=42))

# Evaluating the cross-validated predictions for SVM
svm_accuracy_cv = accuracy_score(y_train, svm_predictions_cv)
svm_precision_cv = precision_score(y_train, svm_predictions_cv)
svm_recall_cv = recall_score(y_train, svm_predictions_cv)
svm_f1_cv = f1_score(y_train, svm_predictions_cv)

print("SVM Accuracy (Cross-Validation):", svm_accuracy_cv)
print("SVM Precision (Cross-Validation):", svm_precision_cv)
print("SVM Recall (Cross-Validation):", svm_recall_cv)
print("SVM F1 Score (Cross-Validation):", svm_f1_cv)

###########################################################
# Creating Random Forest classifier
rf_classifier = RandomForestClassifier()

# Training Random Forest classifier on the training set
rf_classifier.fit(X_train, y_train)

# Making predictions on the test set using Random Forest
rf_predictions_test = rf_classifier.predict(X_test)

# Evaluating Random Forest model on the test set
rf_accuracy_test = accuracy_score(y_test, rf_predictions_test)
rf_precision_test = precision_score(y_test, rf_predictions_test)
rf_recall_test = recall_score(y_test, rf_predictions_test)
rf_f1_test = f1_score(y_test, rf_predictions_test)

# Performing K-fold cross-validation on the training set for Random Forest
rf_predictions_cv = cross_val_predict(rf_classifier, X_train, y_train, cv=KFold(n_splits=10, shuffle=True, random_state=42))

# Evaluating the cross-validated predictions for Random Forest
rf_accuracy_cv = accuracy_score(y_train, rf_predictions_cv)
rf_precision_cv = precision_score(y_train, rf_predictions_cv)
rf_recall_cv = recall_score(y_train, rf_predictions_cv)
rf_f1_cv = f1_score(y_train, rf_predictions_cv)
print("Random Forest Accuracy (Cross-Validation):", rf_accuracy_cv)
print("Random Forest Precision (Cross-Validation):", rf_precision_cv)
print("Random Forest Recall (Cross-Validation):", rf_recall_cv)
print("Random Forest F1 Score (Cross-Validation):", rf_f1_cv)

##############################################
# Creating XGBoost classifier
xgb_classifier = XGBClassifier()

# Training XGBoost classifier on the training set
xgb_classifier.fit(X_train, y_train)

# Making predictions on the test set using XGBoost
xgb_predictions_test = xgb_classifier.predict(X_test)

# Evaluating XGBoost model on the test set
xgb_accuracy_test = accuracy_score(y_test, xgb_predictions_test)
xgb_precision_test = precision_score(y_test, xgb_predictions_test)
xgb_recall_test = recall_score(y_test, xgb_predictions_test)
xgb_f1_test = f1_score(y_test, xgb_predictions_test)

# Performing K-fold cross-validation on the training set for XGBoost
xgb_predictions_cv = cross_val_predict(xgb_classifier, X_train, y_train, cv=KFold(n_splits=10, shuffle=True, random_state=42))

# Evaluating the cross-validated predictions for XGBoost
xgb_accuracy_cv = accuracy_score(y_train, xgb_predictions_cv)
xgb_precision_cv = precision_score(y_train, xgb_predictions_cv)
xgb_recall_cv = recall_score(y_train, xgb_predictions_cv)
xgb_f1_cv = f1_score(y_train, xgb_predictions_cv)
print("XGBoost Accuracy (Cross-Validation):", xgb_accuracy_cv)
print("XGBoost Precision (Cross-Validation):", xgb_precision_cv)
print("XGBoost Recall (Cross-Validation):", xgb_recall_cv)
print("XGBoost F1 Score (Cross-Validation):", xgb_f1_cv)

#########################################################
# Creating k-Nearest Neighbors classifier
knn_classifier = KNeighborsClassifier()

# Creating an ensemble of k-Nearest Neighbors classifiers using Bagging
ensemble_knn_classifier = BaggingClassifier(base_estimator=knn_classifier, n_estimators=10, random_state=42)

# Training ensemble of k-Nearest Neighbors classifiers on the training set
ensemble_knn_classifier.fit(X_train, y_train)

# Making predictions on the test set using the ensemble
ensemble_knn_predictions_test = ensemble_knn_classifier.predict(X_test)

# Evaluating ensemble of k-Nearest Neighbors model on the test set
ensemble_knn_accuracy_test = accuracy_score(y_test, ensemble_knn_predictions_test)
ensemble_knn_precision_test = precision_score(y_test, ensemble_knn_predictions_test)
ensemble_knn_recall_test = recall_score(y_test, ensemble_knn_predictions_test)
ensemble_knn_f1_test = f1_score(y_test, ensemble_knn_predictions_test)

# Performing K-fold cross-validation on the training set for the ensemble
ensemble_knn_predictions_cv = cross_val_predict(ensemble_knn_classifier, X_train, y_train, cv=KFold(n_splits=10, shuffle=True, random_state=42))

# Evaluating the cross-validated predictions for the ensemble
ensemble_knn_accuracy_cv = accuracy_score(y_train, ensemble_knn_predictions_cv)
ensemble_knn_precision_cv = precision_score(y_train, ensemble_knn_predictions_cv)
ensemble_knn_recall_cv = recall_score(y_train, ensemble_knn_predictions_cv)
ensemble_knn_f1_cv = f1_score(y_train, ensemble_knn_predictions_cv)
print("Ensemble (k-NN) Accuracy (Cross-Validation):", ensemble_knn_accuracy_cv)
print("Ensemble (k-NN) Precision (Cross-Validation):", ensemble_knn_precision_cv)
print("Ensemble (k-NN) Recall (Cross-Validation):", ensemble_knn_recall_cv)
print("Ensemble (k-NN) F1 Score (Cross-Validation):", ensemble_knn_f1_cv)

###########################################################################
# Define LSTM model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, input_shape=input_shape))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Define K-fold cross-validation
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True)

# Initialize lists to store evaluation metrics
accuracy_cv = []
precision_cv = []
recall_cv = []
f1_cv = []

# Perform K-fold cross-validation
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    # Reshape input data for LSTM
    X_train_fold = np.reshape(X_train_fold.values, (X_train_fold.shape[0], 1, X_train_fold.shape[1]))
    X_val_fold = np.reshape(X_val_fold.values, (X_val_fold.shape[0], 1, X_val_fold.shape[1]))

    # Create and train LSTM model
    model = create_lstm_model((X_train_fold.shape[1], X_train_fold.shape[2]))
    model.fit(X_train_fold, y_train_fold, epochs=10, batch_size=32, verbose=0)

    # Evaluate model on validation set
    y_pred_prob = model.predict(X_val_fold)
    y_pred = (y_pred_prob > 0.5).astype(int)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_val_fold, y_pred)
    precision = precision_score(y_val_fold, y_pred)
    recall = recall_score(y_val_fold, y_pred)
    f1 = f1_score(y_val_fold, y_pred)

    # Append evaluation metrics to lists
    accuracy_cv.append(accuracy)
    precision_cv.append(precision)
    recall_cv.append(recall)
    f1_cv.append(f1)

# Print average evaluation metrics across folds for LSTM
print("Average validation accuracy for LSTM: ", np.mean(accuracy_cv))
print("Average validation precision for LSTM: ", np.mean(precision_cv))
print("Average validation recall for LSTM: ", np.mean(recall_cv))
print("Average validation F1 score for LSTM: ", np.mean(f1_cv))




SVM Accuracy (Cross-Validation): 0.8856627296587927
SVM Precision (Cross-Validation): 0.8994620040349698
SVM Recall (Cross-Validation): 0.8704848682069639
SVM F1 Score (Cross-Validation): 0.884736232842732
Random Forest Accuracy (Cross-Validation): 0.9215879265091863
Random Forest Precision (Cross-Validation): 0.9154658981748319
Random Forest Recall (Cross-Validation): 0.9303612105434429
Random Forest F1 Score (Cross-Validation): 0.922853453841188
XGBoost Accuracy (Cross-Validation): 0.9517716535433071
XGBoost Precision (Cross-Validation): 0.9475040257648953
XGBoost Recall (Cross-Validation): 0.9573706475756589
XGBoost F1 Score (Cross-Validation): 0.9524117837487859


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/en

Ensemble (k-NN) Accuracy (Cross-Validation): 0.8882874015748031
Ensemble (k-NN) Precision (Cross-Validation): 0.8835792174470815
Ensemble (k-NN) Recall (Cross-Validation): 0.8965180605271721
Ensemble (k-NN) F1 Score (Cross-Validation): 0.8900016152479405
20/20 [==============================] - 1s 3ms/step
Average validation accuracy for LSTM:  0.8310374438073703
Average validation precision for LSTM:  0.837497486804691
Average validation recall for LSTM:  0.8308970405197158
Average validation F1 score for LSTM:  0.8315408573464191


In [116]:
path = '/content/drive/MyDrive/Dataset/Features/Katz_Fd_Features_a_c.csv'
Katz_Fd_Features_a_c = pd.read_csv(path)

path = '/content/drive/MyDrive/Dataset/Features/merged_beta_gamma_a_c.csv'
merged_beta_gamma_a_c = pd.read_csv(path)
# merged_beta_gamma_a_c=merged_beta_gamma_a_c.drop(columns=['X_PS','X_PS_2','Y_PS','Y_PS_2','nd_PS', 'nd_PS_2'], axis=False)


In [117]:
merged_beta_gamma_katz_a_c = pd.concat([merged_beta_gamma_a_c,Katz_Fd_Features_a_c], axis=1)
merged_beta_gamma_katz_a_c.drop('label', axis=1, inplace=True)
merged_beta_gamma_katz_a_c['label']='normal'
merged_beta_gamma_katz_a_c.loc[0:3810, 'label'] = 'addicted'
merged_beta_gamma_katz_a_c

,FP1_PS,FP2_PS,F7_PS,F8_PS,AF1_PS,AF2_PS,FZ_PS,F4_PS,F3_PS,FC6_PS,...,FCZ_PS_2,POZ_PS_2,OZ_PS_2,P2_PS_2,P1_PS_2,CPZ_PS_2,nd_PS_2,Y_PS_2,FP2,label
0,49.125112,75.401561,55.085147,236.773934,24.574587,34.914910,9.395342,15.433209,21.208462,9.854879,...,0.639929,3.175360,7.257440,1.171091,1.727645,0.303052,5.626485,6.090333,1.655864,addicted
1,31.662543,140.818732,107.251892,299.075377,22.347470,53.286704,9.112230,29.795814,17.161432,20.283656,...,0.481474,1.360249,3.002968,0.452312,0.873387,0.169722,4.693551,3.472028,1.367933,addicted
2,33.981550,119.618734,69.705635,404.815675,24.381613,75.511201,9.212180,28.717643,24.864757,23.216713,...,0.403271,1.391440,2.908869,0.790318,1.077771,0.227142,6.266331,3.612442,1.584968,addicted
3,39.844284,101.587017,85.574482,491.630726,27.440661,55.378706,14.178997,42.917834,23.212199,26.417654,...,0.450695,1.121842,2.162650,0.753562,0.999099,0.180620,4.211706,6.950914,1.282758,addicted
4,51.900094,54.809999,113.999959,336.252881,29.900918,53.569130,11.944623,34.313907,17.342946,30.299428,...,1.011451,3.663678,5.646425,1.918202,2.340282,0.437771,7.413797,7.107321,1.461674,addicted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7617,56.398217,64.924927,102.732291,88.351802,48.641285,65.179016,36.246569,81.446588,56.383928,53.427282,...,0.227139,3.721454,4.377724,1.713244,1.729774,0.453551,5.866098,3.061931,1.538744,normal
7618,16.385750,19.103398,36.256479,29.794270,14.541851,18.748430,13.719778,28.332358,13.236784,30.672314,...,0.398802,3.180998,2.880482,2.019003,1.853624,0.785453,2.408728,3.512361,1.716444,normal
7619,53.679662,66.320784,87.491928,83.587297,46.040212,55.310880,36.804056,60.752408,44.290746,36.999455,...,0.132006,1.298873,1.126842,0.855509,0.550932,0.247462,1.363607,3.663259,1.556286,normal
7620,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,normal


In [118]:
# merged_beta_gamma_katz_a_c = merged_beta_gamma_katz_a_c.replace(0, merged_beta_gamma_katz_a_c.mean())
merged_beta_gamma_katz_a_c = merged_beta_gamma_katz_a_c.replace(0, merged_beta_gamma_katz_a_c.mean(numeric_only=True))
# merged_beta_gamma_katz_a_c = merged_beta_gamma_katz_a_c.drop(7620)
# merged_beta_gamma_katz_a_c = merged_beta_gamma_katz_a_c.reset_index(drop=True)
merged_beta_gamma_katz_a_c

,FP1_PS,FP2_PS,F7_PS,F8_PS,AF1_PS,AF2_PS,FZ_PS,F4_PS,F3_PS,FC6_PS,...,FCZ_PS_2,POZ_PS_2,OZ_PS_2,P2_PS_2,P1_PS_2,CPZ_PS_2,nd_PS_2,Y_PS_2,FP2,label
0,49.125112,75.401561,55.085147,236.773934,24.574587,34.914910,9.395342,15.433209,21.208462,9.854879,...,0.639929,3.175360,7.257440,1.171091,1.727645,0.303052,5.626485,6.090333,1.655864,addicted
1,31.662543,140.818732,107.251892,299.075377,22.347470,53.286704,9.112230,29.795814,17.161432,20.283656,...,0.481474,1.360249,3.002968,0.452312,0.873387,0.169722,4.693551,3.472028,1.367933,addicted
2,33.981550,119.618734,69.705635,404.815675,24.381613,75.511201,9.212180,28.717643,24.864757,23.216713,...,0.403271,1.391440,2.908869,0.790318,1.077771,0.227142,6.266331,3.612442,1.584968,addicted
3,39.844284,101.587017,85.574482,491.630726,27.440661,55.378706,14.178997,42.917834,23.212199,26.417654,...,0.450695,1.121842,2.162650,0.753562,0.999099,0.180620,4.211706,6.950914,1.282758,addicted
4,51.900094,54.809999,113.999959,336.252881,29.900918,53.569130,11.944623,34.313907,17.342946,30.299428,...,1.011451,3.663678,5.646425,1.918202,2.340282,0.437771,7.413797,7.107321,1.461674,addicted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7617,56.398217,64.924927,102.732291,88.351802,48.641285,65.179016,36.246569,81.446588,56.383928,53.427282,...,0.227139,3.721454,4.377724,1.713244,1.729774,0.453551,5.866098,3.061931,1.538744,normal
7618,16.385750,19.103398,36.256479,29.794270,14.541851,18.748430,13.719778,28.332358,13.236784,30.672314,...,0.398802,3.180998,2.880482,2.019003,1.853624,0.785453,2.408728,3.512361,1.716444,normal
7619,53.679662,66.320784,87.491928,83.587297,46.040212,55.310880,36.804056,60.752408,44.290746,36.999455,...,0.132006,1.298873,1.126842,0.855509,0.550932,0.247462,1.363607,3.663259,1.556286,normal
7620,92.897973,89.041783,81.298996,88.143716,46.572059,44.647671,28.145790,38.853392,37.093237,47.966618,...,0.661268,2.635775,5.232391,1.594932,1.549560,0.637978,6.253633,7.101868,1.614143,normal


In [119]:
merged_beta_gamma_katz_a_c

,FP1_PS,FP2_PS,F7_PS,F8_PS,AF1_PS,AF2_PS,FZ_PS,F4_PS,F3_PS,FC6_PS,...,FCZ_PS_2,POZ_PS_2,OZ_PS_2,P2_PS_2,P1_PS_2,CPZ_PS_2,nd_PS_2,Y_PS_2,FP2,label
0,49.125112,75.401561,55.085147,236.773934,24.574587,34.914910,9.395342,15.433209,21.208462,9.854879,...,0.639929,3.175360,7.257440,1.171091,1.727645,0.303052,5.626485,6.090333,1.655864,addicted
1,31.662543,140.818732,107.251892,299.075377,22.347470,53.286704,9.112230,29.795814,17.161432,20.283656,...,0.481474,1.360249,3.002968,0.452312,0.873387,0.169722,4.693551,3.472028,1.367933,addicted
2,33.981550,119.618734,69.705635,404.815675,24.381613,75.511201,9.212180,28.717643,24.864757,23.216713,...,0.403271,1.391440,2.908869,0.790318,1.077771,0.227142,6.266331,3.612442,1.584968,addicted
3,39.844284,101.587017,85.574482,491.630726,27.440661,55.378706,14.178997,42.917834,23.212199,26.417654,...,0.450695,1.121842,2.162650,0.753562,0.999099,0.180620,4.211706,6.950914,1.282758,addicted
4,51.900094,54.809999,113.999959,336.252881,29.900918,53.569130,11.944623,34.313907,17.342946,30.299428,...,1.011451,3.663678,5.646425,1.918202,2.340282,0.437771,7.413797,7.107321,1.461674,addicted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7617,56.398217,64.924927,102.732291,88.351802,48.641285,65.179016,36.246569,81.446588,56.383928,53.427282,...,0.227139,3.721454,4.377724,1.713244,1.729774,0.453551,5.866098,3.061931,1.538744,normal
7618,16.385750,19.103398,36.256479,29.794270,14.541851,18.748430,13.719778,28.332358,13.236784,30.672314,...,0.398802,3.180998,2.880482,2.019003,1.853624,0.785453,2.408728,3.512361,1.716444,normal
7619,53.679662,66.320784,87.491928,83.587297,46.040212,55.310880,36.804056,60.752408,44.290746,36.999455,...,0.132006,1.298873,1.126842,0.855509,0.550932,0.247462,1.363607,3.663259,1.556286,normal
7620,92.897973,89.041783,81.298996,88.143716,46.572059,44.647671,28.145790,38.853392,37.093237,47.966618,...,0.661268,2.635775,5.232391,1.594932,1.549560,0.637978,6.253633,7.101868,1.614143,normal


In [120]:
# Assuming 'new_dataframe' is your DataFrame
X = merged_beta_gamma_katz_a_c.iloc[:, :-1]
y = merged_beta_gamma_katz_a_c.iloc[:, -1]

# Encode the target variable if it's categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Splitting the data into training and test sets (80% training, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.20, random_state=42)

In [122]:

# Creating SVM classifier pipeline
svm_classifier = make_pipeline(StandardScaler(), SVC())

# Training SVM classifier on the training set
svm_classifier.fit(X_train, y_train)

# Making predictions on the test set using SVM
svm_predictions_test = svm_classifier.predict(X_test)

# Evaluating SVM model on the test set
svm_accuracy_test = accuracy_score(y_test, svm_predictions_test)
svm_precision_test = precision_score(y_test, svm_predictions_test)
svm_recall_test = recall_score(y_test, svm_predictions_test)
svm_f1_test = f1_score(y_test, svm_predictions_test)

# Performing K-fold cross-validation on the training set for SVM
svm_predictions_cv = cross_val_predict(svm_classifier, X_train, y_train, cv=KFold(n_splits=10, shuffle=True, random_state=42))

# Evaluating the cross-validated predictions for SVM
svm_accuracy_cv = accuracy_score(y_train, svm_predictions_cv)
svm_precision_cv = precision_score(y_train, svm_predictions_cv)
svm_recall_cv = recall_score(y_train, svm_predictions_cv)
svm_f1_cv = f1_score(y_train, svm_predictions_cv)

print("SVM Accuracy (Cross-Validation):", svm_accuracy_cv)
print("SVM Precision (Cross-Validation):", svm_precision_cv)
print("SVM Recall (Cross-Validation):", svm_recall_cv)
print("SVM F1 Score (Cross-Validation):", svm_f1_cv)
print("\n")
###########################################################
# Creating Random Forest classifier
rf_classifier = RandomForestClassifier()

# Training Random Forest classifier on the training set
rf_classifier.fit(X_train, y_train)

# Making predictions on the test set using Random Forest
rf_predictions_test = rf_classifier.predict(X_test)

# Evaluating Random Forest model on the test set
rf_accuracy_test = accuracy_score(y_test, rf_predictions_test)
rf_precision_test = precision_score(y_test, rf_predictions_test)
rf_recall_test = recall_score(y_test, rf_predictions_test)
rf_f1_test = f1_score(y_test, rf_predictions_test)

# Performing K-fold cross-validation on the training set for Random Forest
rf_predictions_cv = cross_val_predict(rf_classifier, X_train, y_train, cv=KFold(n_splits=10, shuffle=True, random_state=42))

# Evaluating the cross-validated predictions for Random Forest
rf_accuracy_cv = accuracy_score(y_train, rf_predictions_cv)
rf_precision_cv = precision_score(y_train, rf_predictions_cv)
rf_recall_cv = recall_score(y_train, rf_predictions_cv)
rf_f1_cv = f1_score(y_train, rf_predictions_cv)
print("Random Forest Accuracy (Cross-Validation):", rf_accuracy_cv)
print("Random Forest Precision (Cross-Validation):", rf_precision_cv)
print("Random Forest Recall (Cross-Validation):", rf_recall_cv)
print("Random Forest F1 Score (Cross-Validation):", rf_f1_cv)
print("\n")
##############################################
# Creating XGBoost classifier
xgb_classifier = XGBClassifier()

# Training XGBoost classifier on the training set
xgb_classifier.fit(X_train, y_train)

# Making predictions on the test set using XGBoost
xgb_predictions_test = xgb_classifier.predict(X_test)

# Evaluating XGBoost model on the test set
xgb_accuracy_test = accuracy_score(y_test, xgb_predictions_test)
xgb_precision_test = precision_score(y_test, xgb_predictions_test)
xgb_recall_test = recall_score(y_test, xgb_predictions_test)
xgb_f1_test = f1_score(y_test, xgb_predictions_test)

# Performing K-fold cross-validation on the training set for XGBoost
xgb_predictions_cv = cross_val_predict(xgb_classifier, X_train, y_train, cv=KFold(n_splits=10, shuffle=True, random_state=42))

# Evaluating the cross-validated predictions for XGBoost
xgb_accuracy_cv = accuracy_score(y_train, xgb_predictions_cv)
xgb_precision_cv = precision_score(y_train, xgb_predictions_cv)
xgb_recall_cv = recall_score(y_train, xgb_predictions_cv)
xgb_f1_cv = f1_score(y_train, xgb_predictions_cv)
print("XGBoost Accuracy (Cross-Validation):", xgb_accuracy_cv)
print("XGBoost Precision (Cross-Validation):", xgb_precision_cv)
print("XGBoost Recall (Cross-Validation):", xgb_recall_cv)
print("XGBoost F1 Score (Cross-Validation):", xgb_f1_cv)
print("\n")
#########################################################
# Creating k-Nearest Neighbors classifier
knn_classifier = KNeighborsClassifier()

# Creating an ensemble of k-Nearest Neighbors classifiers using Bagging
ensemble_knn_classifier = BaggingClassifier(base_estimator=knn_classifier, n_estimators=10, random_state=42)

# Training ensemble of k-Nearest Neighbors classifiers on the training set
ensemble_knn_classifier.fit(X_train, y_train)

# Making predictions on the test set using the ensemble
ensemble_knn_predictions_test = ensemble_knn_classifier.predict(X_test)

# Evaluating ensemble of k-Nearest Neighbors model on the test set
ensemble_knn_accuracy_test = accuracy_score(y_test, ensemble_knn_predictions_test)
ensemble_knn_precision_test = precision_score(y_test, ensemble_knn_predictions_test)
ensemble_knn_recall_test = recall_score(y_test, ensemble_knn_predictions_test)
ensemble_knn_f1_test = f1_score(y_test, ensemble_knn_predictions_test)

# Performing K-fold cross-validation on the training set for the ensemble
ensemble_knn_predictions_cv = cross_val_predict(ensemble_knn_classifier, X_train, y_train, cv=KFold(n_splits=10, shuffle=True, random_state=42))

# Evaluating the cross-validated predictions for the ensemble
ensemble_knn_accuracy_cv = accuracy_score(y_train, ensemble_knn_predictions_cv)
ensemble_knn_precision_cv = precision_score(y_train, ensemble_knn_predictions_cv)
ensemble_knn_recall_cv = recall_score(y_train, ensemble_knn_predictions_cv)
ensemble_knn_f1_cv = f1_score(y_train, ensemble_knn_predictions_cv)
print("Ensemble (k-NN) Accuracy (Cross-Validation):", ensemble_knn_accuracy_cv)
print("Ensemble (k-NN) Precision (Cross-Validation):", ensemble_knn_precision_cv)
print("Ensemble (k-NN) Recall (Cross-Validation):", ensemble_knn_recall_cv)
print("Ensemble (k-NN) F1 Score (Cross-Validation):", ensemble_knn_f1_cv)
print("\n")
###########################################################################
# Define LSTM model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, input_shape=input_shape))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Define K-fold cross-validation
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True)

# Initialize lists to store evaluation metrics
accuracy_cv = []
precision_cv = []
recall_cv = []
f1_cv = []

# Perform K-fold cross-validation
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    # Reshape input data for LSTM
    X_train_fold = np.reshape(X_train_fold.values, (X_train_fold.shape[0], 1, X_train_fold.shape[1]))
    X_val_fold = np.reshape(X_val_fold.values, (X_val_fold.shape[0], 1, X_val_fold.shape[1]))

    # Create and train LSTM model
    model = create_lstm_model((X_train_fold.shape[1], X_train_fold.shape[2]))
    model.fit(X_train_fold, y_train_fold, epochs=10, batch_size=32, verbose=0)

    # Evaluate model on validation set
    y_pred_prob = model.predict(X_val_fold)
    y_pred = (y_pred_prob > 0.5).astype(int)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_val_fold, y_pred)
    precision = precision_score(y_val_fold, y_pred)
    recall = recall_score(y_val_fold, y_pred)
    f1 = f1_score(y_val_fold, y_pred)

    # Append evaluation metrics to lists
    accuracy_cv.append(accuracy)
    precision_cv.append(precision)
    recall_cv.append(recall)
    f1_cv.append(f1)

# Print average evaluation metrics across folds for LSTM
print("Average validation accuracy for LSTM: ", np.mean(accuracy_cv))
print("Average validation precision for LSTM: ", np.mean(precision_cv))
print("Average validation recall for LSTM: ", np.mean(recall_cv))
print("Average validation F1 score for LSTM: ", np.mean(f1_cv))




SVM Accuracy (Cross-Validation): 0.8868295883221257
SVM Precision (Cross-Validation): 0.8986622073578595
SVM Recall (Cross-Validation): 0.8741054001301236
SVM F1 Score (Cross-Validation): 0.8862137203166227


Random Forest Accuracy (Cross-Validation): 0.9170083647695588
Random Forest Precision (Cross-Validation): 0.9133934320669672
Random Forest Recall (Cross-Validation): 0.9229017566688354
Random Forest F1 Score (Cross-Validation): 0.9181229773462783


XGBoost Accuracy (Cross-Validation): 0.9548958504182384
XGBoost Precision (Cross-Validation): 0.9545306917830464
XGBoost Recall (Cross-Validation): 0.9560832791151594
XGBoost F1 Score (Cross-Validation): 0.9553063546237608




/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/en

Ensemble (k-NN) Accuracy (Cross-Validation): 0.8917500410037723
Ensemble (k-NN) Precision (Cross-Validation): 0.8886027044430135
Ensemble (k-NN) Recall (Cross-Validation): 0.8978529603122967
Ensemble (k-NN) F1 Score (Cross-Validation): 0.8932038834951457


20/20 [==============================] - 1s 4ms/step
Average validation accuracy for LSTM:  0.8277816361140273
Average validation precision for LSTM:  0.8195822383866469
Average validation recall for LSTM:  0.8459252522032182
Average validation F1 score for LSTM:  0.8318223692497482
